<a href="https://colab.research.google.com/github/toche7/PromptEngforDeveloper/blob/main/PromptLab1_GuidelinesforPrompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guidelines for Prompting
ในบทนี้ เราจะได้ฝึกฝนหลักการสองประการในการใช้ prompt และเทคนิควิธีที่เกี่ยวข้องเพื่อเขียน  prompt ที่มีประสิทธิภาพสำหรับ  LLM (large language models).




## Setup
#### Load the API key and relevant Python libaries.

ในกรณีที่ใช้ Google Colab จำเป็นที่จะต้องติดตั้ง Library ของ Groq ก่อนตามตัวอย่างด้านล่าง และ
ใน Course นี้ผู้เรียนจะต้องไปเอา API Key มาจากเวปไซท์ของ Groq

In [ ]:
# !pip -q install python-dotenv
!pip -q install groq

In [ ]:
#=========VSCode=====================================================================
# from dotenv import find_dotenv, load_dotenv
# _ = load_dotenv(find_dotenv())
# import os
# secret_key = os.environ.get("GROQ_API_KEY")
#========google.colab======================================================================
from google.colab import userdata
secret_key = userdata.get('GROQ_API_KEY')



In [ ]:
from groq import Groq

clientGroq = Groq(
    api_key= secret_key,
)

#### helper function
ตลอดการอบรมนี้ เราจะใช้ Model llama3 ผ่านทางบริการของ Groq 

Function ด้านล่างจะช่วยให้เราเขียนโปรแกรมได้ง่ายขึ้น

In [ ]:
def get_completion(prompt, model= "llama3-70b-8192", clientGroq = clientGroq):
    messages = [{"role": "user", "content": prompt}]
    response = clientGroq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens= 4000,
        top_p = 0.5
    )
    return response.choices[0].message.content

## หลักการเขียน  Prompt
- **หลักการที่ 1: เขียนให้ชัดเจนและจำเพาะเจาะจง**
- **หลักการที่ 2: ให้เวลาในการคิด**




## Principle 1: เขียนให้ชัดเจนและจำเพาะเจาะจง

#### Tactic 1: ใช้สัญลักษณ์พิเศษในการแบ่งแยกข้อความ แต่ละส่วนให้ชัดเจน
- ทั้งนี้จะเป็นสัญลักษณ์อะไรก็ได้เช่น ```, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

#### Tactic 2:  กำหนดรูปแบบของ output ที่ต้องการ
- JSON, HTML

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON  with the following keys:
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

#### Tactic 3: สั่งให้ model ตรวจสอบเงื่อนไขว่าเป็นไปตามที่กำหนดหรือไม่

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

#### Tactic 4: "Few-shot" prompting หมายถึงการทำให้ดูเป็นตัวอย่างที่เราต้องการ และให้ทำตาม

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

## Principle 2: "ให้เวลาคิด"

#### Tactic 1: กำหนดขั้นตอนในการทำงานใดงานหนึ่งให้ชัดเจน อย่าคิดลัดนะ  

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Thai.
3 - List each name in the Thai summary.
4 - Output a json object that contains the following \
keys: thai_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

#### สั่งให้แสดงในรูปแบบที่ต้องการได้

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into Thai.
3 - List each name in the Thai summary.
4 - Output a json object that contains the
  following keys: thai_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)

#### Tactic 2:  สั่งให้แบบจำลองแสดงวิธีการคิดหรือทำงานออกมาก่อนที่จะสรุปผล

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

- เราจะพบว่าคำตอบที่ได้จะยังไม่ถูกต้อง
- เราสามารถแก้ไขได้โดยการสั้งให้ model ดำเนินการเป็นขั้นตอนก่อนที่จะตอบคำถามนั้น

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
```
....
```

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```
"""
response = get_completion(prompt)
print(response)

## Model Limitations: Hallucinations
- Let try about ITM.

In [ ]:
prompt = f"""
Tell me about  ITM, Faculy of Engineering, Mahidol University
"""
response = get_completion(prompt)
print(response)

## Try experimenting on your own!

#### Ex 1. จงเขียน Prompt เพื่อตรวจข้อสอบต่อไปนี้ว่าถูกต้องหรือไม่
**question**: รูปสามเหลี่ยมรูปหนึ่งมีพื้นที่ 32 ตารางเซนติเมตร มีฐานยาว 8 เซนติเมตร รูปสามเหลี่ยมรูปนี้สูงกี่เซนติเมตร<br>
**answer** : 8  เซนติเมตร


In [ ]:
question = 'รูปสามเหลี่ยมรูปหนึ่งมีพื้นที่ 32 ตารางเซนติเมตร มีฐานยาว 8 เซนติเมตร รูปสามเหลี่ยมรูปนี้สูงกี่เซนติเมตร'
answer = '8 เซนติเมตร'

#### Ex 2.
จงเขียน Prompt เพื่อสรุปข่าวต่อไปนี้สำหรับใส่ลงไปใน Facebook

"เมื่อวันที่ 22 เมษายน 2567 สถาบันคุณวุฒิวิชาชีพ นำโดย คุณวรากร ธนูสนธิ์ หัวหน้ากลุ่มงานและคณะผู้ตรวจประเมิน ได้เข้าตรวจประเมิน “ศูนย์ฝึกอบรมฐานสมรรถนะตามมาตรฐานคุณวุฒิวิชาชีพ” ที่ดำเนินงานโดยกลุ่มสาขาวิชาเทคโนโลยีการจัดการระบบสารสนเทศ คณะวิศวกรรมศาสตร์ มหาวิทยาลัยมหิดล ตามคำขอการรับรองให้เป็นองค์กรที่ทำหน้าที่ให้บริการฝึกอบรมตามข้อกำหนดของสถาบันฯ โดยในงานตรวจประเมินครั้งนี้ มี ผศ.ดร. ทวีศักดิ์ สมานชื่น หัวหน้าโครงการศูนย์ฝึกฯ ได้ให้การต้อนรับและนำเสนอรายงานต่อคณะผู้ตรวจ พร้อมกับบุคลากรที่เกี่ยวข้อง อีกทั้งมีการตรวจสถานที่ที่ใช้ในการฝึกอบรม ตรวจความพร้อมเครื่องมือ และอุปกรณ์ที่เกี่ยวข้องในการฝึกอบรม จากการตรวจประเมิน สถาบันคุณวุฒิวิชาชีพได้สรุปผลการประเมินพร้อมข้อเสนอแนะเพิ่มเติมที่ให้องค์กรที่ทำหน้าที่ให้บริการฝึกอบรม การตรวจประเมินครั้งนี้ ถือเป็นการยกระดับมาตรฐานการฝึกอบรม หลักสูตร สาขาวิชาชีพอุตสาหกรรมดิจิทัล เป็นมาตรฐานองค์ฝึกอบรมตามข้อกำหนดของสถาบันฯ"

